leetcode_dataset.csv 파일 불러오기 및 수정

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
data=pd.read_csv('leetcode_dataset.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1825 entries, 0 to 1824
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 1825 non-null   int64  
 1   title              1825 non-null   object 
 2   description        1825 non-null   object 
 3   is_premium         1825 non-null   int64  
 4   difficulty         1825 non-null   object 
 5   solution_link      987 non-null    object 
 6   acceptance_rate    1825 non-null   float64
 7   frequency          1825 non-null   float64
 8   url                1825 non-null   object 
 9   discuss_count      1825 non-null   int64  
 10  accepted           1825 non-null   object 
 11  submissions        1825 non-null   object 
 12  companies          1749 non-null   object 
 13  related_topics     1571 non-null   object 
 14  likes              1825 non-null   int64  
 15  dislikes           1825 non-null   int64  
 16  rating             1825 

In [2]:
#인덱스 추출
leetcode = data[['id','title','description','related_topics','python_template','c_template']]
leetcode.head(10)

,id,title,description,related_topics,python_template,c_template
0,1,Two Sum,Given an array of integers `nums` and an integ...,"Array,Hash Table",Create Python code based on outline below.\ncl...,Create c code based on outline below.\n/**\n *...
1,2,Add Two Numbers,You are given two non-empty linked lists repre...,"Linked List,Math,Recursion",Create Python code based on outline below.\n# ...,Create c code based on outline below.\n/**\n *...
2,3,Longest Substring Without Repeating Characters,"Given a string `s`, find the length of the lon...","Hash Table,Two Pointers,String,Sliding Window",Create Python code based on outline below.\ncl...,Create c code based on outline below.\nint len...
3,4,Median of Two Sorted Arrays,Given two sorted arrays `nums1` and `nums2` of...,"Array,Binary Search,Divide and Conquer",Create Python code based on outline below.\ncl...,Create c code based on outline below.\ndouble ...
4,5,Longest Palindromic Substring,"Given a string `s`, return the longest palindr...","String,Dynamic Programming",Create Python code based on outline below.\ncl...,Create c code based on outline below.\nchar * ...
5,6,ZigZag Conversion,"The string `""PAYPALISHIRING""` is written in a ...",String,Create Python code based on outline below.\ncl...,Create c code based on outline below.\nchar * ...
6,7,Reverse Integer,"Given a signed 32-bit integer `x`, return `x` ...",Math,Create Python code based on outline below.\ncl...,Create c code based on outline below.\nint rev...
7,8,String to Integer (atoi),"Implement the `myAtoi(string s)` function, whi...","Math,String",Create Python code based on outline below.\ncl...,Create c code based on outline below.\nint myA...
8,9,Palindrome Number,"Given an integer `x`, return `true` if `x` is ...",Math,Create Python code based on outline below.\ncl...,Create c code based on outline below.\nbool is...
9,10,Regular Expression Matching,Given an input string (`s`) and a pattern (`p`...,"String,Dynamic Programming,Backtracking",Create Python code based on outline below.\ncl...,Create c code based on outline below.\nbool is...


chat GPT에게 input할 데이터 파싱

In [3]:
# 각 행(문제)마다 description열의 내용에서 d, e, c를 추출하여 저장할 변수 초기화
d = []
e = []
c = []
pcode = []
ccode=[]

# 각 행(문제)을 순회하며 d, e, c 값을 추출하여 저장
for index, row in data.iterrows():
    description = row['description']
    example_start_index = description.find("Example 1")
    constraints_start_index = description.find("Constraints")
    
    # "Example 1" 문자열이 나오기 전까지 description에서 d 값 추출
    if example_start_index != -1:
        d.append(description[:example_start_index])
    else:
        d.append(description)
    
    # "Example 1" 문자열이 나오기 시작해서 "Constraints" 문자열이 나오기 전까지 description에서 e 값 추출
    if example_start_index != -1 and constraints_start_index != -1:
        e.append(description[example_start_index:constraints_start_index])
    else:
        e.append("")
    
    # "Constraints" 문자열이 나오기 시작하는 내용부터 description에서 c 값 추출
    if constraints_start_index != -1:
        c.append(description[constraints_start_index:])
    else:
        c.append("")
    
    # python_template의 내용은 변수 pcode에 저장
    pcode.append(row['python_template'])

    #c_template의 내용은 변수 ccode에 저장
    ccode.append(row['c_template'])

chat GPT와 통신

In [ ]:
# python 코드 생성
import os
import openai
import json
import time

openai.api_key = "YOUR-KEY"

results = []

def count_tokens(text):
    return len(openai.Completion.create(prompt=text, max_tokens=1)["usage"]["total_tokens"])

def extract_class_code(result):
    start_index = result.find("class")
    if start_index == -1:
        return result
    end_index = result.find("```", start_index) if '```' in result else len(result)
    return result[start_index:end_index]

def fetch_code(messages, max_retries=3, delay=60):
    retries = 0
    while retries < max_retries:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=messages
            )
            code = response["choices"][0]["message"]["content"]
            class_code = extract_class_code(code)
            
            token_count = response["usage"]["total_tokens"]
            if token_count >= 4000:
                time.sleep(60)
            elif "class Solution" in class_code:
                return class_code
            else:
                time.sleep(40)
        except Exception as e:
            print(f"Error occurred: {e}. Retrying in {delay} seconds.")
            retries += 1
            time.sleep(delay)
    
    # If we reach here, it means all retries have failed.
    raise Exception("Failed to fetch code after multiple retries.")

for i in range(10,20):
    first_results = []
    second_results = []
    third_results = []

    messages = [
        {"role": "user", "content": "Please implement the following code. I don't need an explanation."},
        {"role": "user", "content": f"Description: {d[i]}"},
        {"role": "user", "content": f"Code outline: {pcode[i]}"}
    ]

    for _ in range(10):
        first_results.append(fetch_code(messages))
        
        messages_second = messages + [{"role": "user", "content": f"Constraints: {c[i]}"}]
        second_results.append(fetch_code(messages_second))
        
        messages_third = messages_second + [{"role": "user", "content": f"Example: {e[i]}"}]
        third_results.append(fetch_code(messages_third))

    result_obj = {
        "num": i + 1,
        "title":leetcode['title'][i],
        "description":d[i],
        "constraints":c[i],
        "category":leetcode['related_topics'][i],
        "python_template": leetcode['python_template'][i],
        "first_result": first_results,
        "second_result": second_results,
        "third_result": third_results
    }
    results.append(result_obj)
    print(i+1,"저장")

filename = "11-20 python-results.json"
with open(filename, mode='w', encoding="utf-8-sig") as file:
    json.dump(results, file, ensure_ascii=False, indent=2)

print(f"모든 문제의 결과가 JSON 파일로 저장되었습니다.")


In [ ]:
#c 코드 생성
import os
import openai
import json
import time

openai.api_key = "YOUR-KEY"

# 결과를 저장할 리스트 초기화
results = []
def extract_code(result):
    # Set data types in C
    data_types = ['char ', 'int ', 'bool ', 'struct ', 'void ', 'double ']
    
    # Find the starting index of each data type in the result, and get the minimum index (excluding -1)
    start_index = min((result.find(data_type) for data_type in data_types if result.find(data_type) != -1), default=-1)
    
    # If there is no data type, return the whole string
    if start_index == -1:
        return result
    
    # Find the end index of "}"
    end_index = result.rfind("}")
    
    # If there is no "}", return the whole string
    if end_index == -1:
        return result
    
    # Extract the substring from the start index to end index
    return result[start_index:end_index+1]



# 각 문제에 대해서 반복 실행
for i in range(100):
    first_results = []
    second_results = []
    third_results = []
    for _ in range(10):
        # first result
        messages = [
            {"role": "user", "content": "다음에 해당하는 코드를 구현해줘. 설명은 필요없어."},
            {"role": "user", "content": f"description: {d[i]}"},
            {"role": "user", "content": f"c otline: {ccode[i]}"},
        ]

        # Send the messages to ChatGPT to get the title and description result
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )

        first_result = response["choices"][0]["message"]["content"]
        first_result_class = extract_code(first_result)
        first_results.append(first_result_class)

        #second result
        messages.append({"role": "user", "content": f"Constraints: {c[i]}"})
        # Send the messages to ChatGPT to get the title and description result
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )

        second_result = response["choices"][0]["message"]["content"]
        second_result_class = extract_code(second_result)
        second_results.append(second_result_class)

        #third result
        messages.append({"role": "user", "content": f"example: {e[i]}"})
        # Send the messages to ChatGPT to get the title and description result
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )

        third_result = response["choices"][0]["message"]["content"]
        third_result_class = extract_code(third_result)
        third_results.append(third_result_class)

    # 결과를 JSON 형식으로 저장
    result_obj = {
        "num": i + 1,
        "title":leetcode['title'][i],
        "description":d[i],
        "constraints":c[i],
        "category":leetcode['related_topics'][i],
        "python_template": leetcode['python_template'][i],
        "first_result": first_results,
        "second_result": second_results,
        "third_result": third_results
    }
    results.append(result_obj)
    print(i+1,"저장")
    time.sleep(20)
# 모든 결과를 하나의 JSON 파일에 저장
filename = "c-results.json"
with open(filename, mode='w', encoding="utf-8-sig") as file:
    json.dump(results, file, ensure_ascii=False, indent=2)

# 결과 출력
print(f"모든 문제의 결과가 JSON 파일로 저장되었습니다.")
